#### Builder Pattern

* 把构造一个类，或者几个类的步骤，过程封装在一个Builder class中
* 并且用另一个Dirctor 去统一调配

在这一点上，建造者模式和工厂模式的差别并不太明确。主要的区别在于工厂模式以单个步
骤创建对象，而建造者模式以多个步骤创建对象，并且几乎始终会使用一个指挥者。

另一个区别是，在工厂模式下，会立即返回一个创建好的对象；而在建造者模式下，仅在需
要时客户端代码才显式地请求指挥者返回最终的对象
* Actor:
    * Builder
    * Directer

In [134]:
import functools

In [1]:
class Computer:

    def __init__(self, serial_number):
        self.serial = serial_number
        self.memory = None
        self.hdd = None
        self.gpu = None

    def __str__(self):
        return str(self.__dict__)


class ComputerBuiler:

    """ Core is that use a BUILDER class to encapsulate 
    each steps to build the target class
    """
    
    def __init__(self):
        self.computer = Computer("AG23386193")

    def configure_memory(self, amount):
        self.computer.memory = amount

    def configure_hdd(self, amount):
        self.computer.hdd = amount

    def configure_gpu(self, gpu_model):
        self.computer.gpu = gpu_model


class HardwareEngineer:

    """ Actor: Director. 
    Call Builder in the right way
    """
    
    def __init__(self):
        self.builder = None

    def construct_computer(self, memory, hdd, gpu):
        self.builder = ComputerBuiler()
        [step for step in (self.builder.configure_memory(memory),
                           self.builder.configure_hdd(hdd),
                           self.builder.configure_gpu(gpu))]

    @property
    def computer(self):
        return self.builder.computer

In [2]:
engineer = HardwareEngineer()
engineer.construct_computer(hdd=500, memory=8, gpu="GeForce GTX 650 Ti")
print(engineer.computer)

{'gpu': 'GeForce GTX 650 Ti', 'serial': 'AG23386193', 'hdd': 500, 'memory': 8}


##### 需求复杂：除了构造过程是分步的，其可构造的种类也是多种

In [3]:
from enum import Enum
import time

STEP_DELAY = 1
PizzaProgress = Enum('PizzaProgress', 'queued preparation baking ready')
PizzaDough = Enum('PizzaDough', 'thin thick')
PizzaSauce = Enum('PizzaSauce', 'tomato creme_fraiche')

In [4]:
class Pizza:

    def __init__(self, name):
        self.name = name
        self.dough = None
        self.sauce = None
        self.topping = []

    def __str__(self):
        return self.name

    def prepare_dough(self, dough):
        self.dough = dough
        print("preparing the {} dough of your {}...").format(
            self.dough.name, self)
        time.sleep(STEP_DELAY)
        print("done with the {} dough").format(self.dough.name)


class MargaritaBuilder:

    def __init__(self):
        self.pizza = Pizza('magarita')
        self.progress = PizzaProgress.queued
        self.baking_time = 5

    def prepare_dough(self):
        self.progress = PizzaProgress.preparation
        self.pizza.prepare_dough(PizzaDough.thin)

    def add_sauce(self):
        print('adding the tomato sauce to your margarita...')
        self.pizza.sauce = PizzaSauce.tomato
        time.sleep(STEP_DELAY)
        print('done with the tomato sauce')

    def add_topping(self):
        pass

    def bake(self):
        self.progress = PizzaProgress.baking
        print("baking your margarita for {} seconds".format(self.baking_time))
        time.sleep(STEP_DELAY)
        print("your margarita is ready")


class CreamyBaconBuilder:
    def __init__(self):
        self.pizza=Pizza('creamy bacon')
        self.progress=PizzaProgress.queued
        self.baking_time=7

    def prepare_dough(self):
        self.progress=PizzaProgress.preparation
        self.pizza.prepare_dough(PizzaDough.thick)

    def add_sauce(self):
        print('adding the creme fraiche sauce to your creamy bacon...')
        self.pizza.sauce=PizzaSauce.creme_fraiche
        time.sleep(STEP_DELAY)
        print('done with the creme fraiche sauce')

    def add_topping(self):
        pass

    def bake(self):
        self.progress = PizzaProgress.baking
        print("baking your margarita for {} seconds".format(self.baking_time))
        time.sleep(STEP_DELAY)
        print("your creamy bacon is ready")


In [5]:
class Waiter:
    def __init__(self):
        self.builder = None
    
    def construct_pizza(self, builder):
        self.builder = builder
        [step() for step in (builder.prepare_dough, builder.add_sauce, builder.add_topping, builder.bake)]
        
    @property
    def pizza(self):
        return self.builder.pizza

In [6]:
builders = dict(m=MargaritaBuilder, c=CreamyBaconBuilder)

input = "m"
waiter = Waiter()
waiter.construct_pizza(builders[input]())

preparing the thin dough of your magarita...
done with the thin dough
adding the tomato sauce to your margarita...
done with the tomato sauce
baking your margarita for 5 seconds
your margarita is ready


In [90]:
input = "c"
waiter = Waiter()
waiter.construct_pizza(builders[input]())

preparing the thick dough of your creamy bacon...
done with the thick dough
adding the creme fraiche sauce to your creamy bacon...
done with the creme fraiche sauce
baking your margarita for 7 seconds
your creamy bacon is ready


#### Adapter Pattern
* 把不同的接口，统一到一个
* 用一个适配器，而不需要用继承把不同的类硬统一

Computer has execute(), other two synthesizer and human dont. 

In [121]:
class Computer:

    def __init__(self, name):
        self.name = name

    def __str__(self):
        return "the {} computer".format(self.name)

    def execute(self):
        return "excutes a program"


class Synthesizer:

    def __init__(self, name):
        self.name = name

    def __str__(self):
        return "the {} synthesizer".format(self.name)

    def play(self):
        return "is playing the electronic song"


class Human:

    def __init__(self, name):
        self.name = name

    def __str__(self):
        return "{} the human".format(self.name)

    def speak(self):
        return "says hello"

!!!

In [123]:
class Adapter:

    def __init__(self, obj, adapted_methods):
        self.obj = obj
        self.__dict__.update(adapted_methods)

    def __str__(self):
        return str(self.obj)

In [118]:
objects = [Computer("Asus")]

synth = Synthesizer('moog')
adapted_synth = Adapter(synth, dict(execute=synth.play))

human = Human("Bob")
adapted_human = Adapter(human, dict(execute=human.speak))

objects.extend([adapted_synth, adapted_human])

In [120]:
[o.execute() for o in objects]

['excutes a program', 'is playing the electronic song', 'says hello']

#### Decorator Pattern

当我们想给对象额外增加功能时：
* 如果合理，可以直接将功能添加到对象所属的类
* 使用组合
* 使用继承

通常应该优先选择组合，因为继承使得代码更难复用，继承关系是静态的，并且应用于整个类以及这个类的所有实例

In [163]:
def memoize(fn):
    known = dict()

    @functools.wraps(fn)
    def memoizer(*args):
        if args not in known:
            known[args] = fn(*args)
        return known[args]

    return memoizer

In [161]:
@memoize
def fibonacci(n):
    assert n >= 0, "n must be possive"
    return n if n in (0,1) else fibonacci(n-1) + fibonacci(n-2)

In [162]:
from timeit import Timer

t = Timer()
fibonacci(100)
t.timeit()

0.012418985366821289

#### Facade Pattern 外观模式

* 封装复杂的内部系统，仅仅给客户一个外观的角色
* 注意是把不同的内部系统，组装封装成一个统一的外部对象。e.g 计算机（处理器，键盘...), 客服（法务，会计，...)
* 在现实中，外观模式相当常见。当你致电一个银行或公司，通常是先被连线到客服部门，客服职员在你和业务部门


In [13]:
from abc import ABCMeta, abstractmethod

In [34]:
State = Enum('State', 'new running sleep restart zombie')

class Server():
    
    @abstractmethod
    def __init__(self):
        pass
    
    def __str__(self):
        return self.name
    
    @abstractmethod
    def boot(self):
        pass
    
    @abstractmethod
    def kill(self, restart=True):
        pass

In [43]:
class FileServer(Server):

    def __init__(self):
        self.name = 'FileServer'
        self.state = State.new

    def boot(self):
        print("booting the {}".format(self))
        self.state = State.running

    def kill(self, restart=True):
        print("Killing {}".format(self))
        self.state = State.restart if restart else State.sleep

    def create_file(self, user, name, permission):
        print("creating files")


class ProcessServer(Server):

    def __init__(self):
        self.name = 'ProcessServer'
        self.state = State.new

    def boot(self):
        print("booting the {}".format(self))
        self.state = State.running

    def kill(self, restart=True):
        print("Killing {}".format(self))
        self.state = State.restart if restart else State.sleep

    def create_process(self, user, name):
        print("Creating process")

In [36]:
class OperatingSystem:
    '''Facade
    Dont care how function implemented. 
    Provide a unified interface to users.
    User would think create_file, or create_process, coming done by os

    Attributes:
        fs (TYPE): file server
        ps (TYPE): process server
    '''
    
    def __init__(self):
        self.fs = FileServer()
        self.ps = ProcessServer()
        
    def start(self):
        [i.boot() for i in (self.fs, self.ps)]
        
    def create_file(self, user, name, permissions):
        return self.fs.create_file(user, name, permissions)
    
    def create_process(self, user, name):
        return self.ps.create_process(user, name)


In [42]:
os = OperatingSystem()
os.start()

booting the FileServer
booting the ProcessServer


In [41]:
os.create_file("root", "a_file", "root")
os.create_process("root", "a_process")

creating files
Creating process


SOLID
* S Single responsibility
* open close

#### 享元模式 ???

### MVC Pattern

### Proxy/Surrogate Pattern

类似于包装类，代理调用底层函数。调用前进行验证，等行为

In [59]:
class LazyProperty(object):
    def __init__(self, method):
        self.method = method
        self.method_name = method.__name__
        print("method name: ", self.method_name)
    
    def __get__(self, obj, cls):
        if not obj:
            return None

        print("calling with: ", obj)
        value = self.method(obj)
        
        setattr(obj, self.method_name, value)
        return value

In [60]:
class Blob(object):
    lazySum = LazyProperty(sum)
    
    def __init__(self, data):
        self.data = data
    
    def __iter__(self):
        return self.data.__iter__()

('method name: ', 'sum')


In [67]:
b = Blob([1, 2, 3, 4])
b.lazySum

('calling with: ', <__main__.Blob object at 0x7f9fa89c7710>)


10

In [68]:
class Test:

    @LazyProperty
    def resource(self):
        self._resource = tuple(range(5))
        return self._resource

('method name: ', 'resource')


In [65]:
Test().resource

('calling with: ', <__main__.Test instance at 0x7f9fa8946248>)


(0, 1, 2, 3, 4)

* Example

In [70]:
class SensitiveInfo:
    
    def __init__(self):
        self.users = ['nick', 'tom', 'ben', 'mike']
        
    def read(self):
        print("users are: ".format(','.join(self.users)))
    
    def add(self, user):
        self.users.append(user)
        print("Added user {}".format(user))

In [84]:
class Info:

    def __init__(self):
        self.protected = SensitiveInfo()
        self.secret = "deadbeef"
        
    def read(self):
        return self.protected.read()
    
    def add(self, user):
        sec = raw_input("What is password")
        if sec == self.secret:
            self.protected.add(user)  
        else:
            print("wrong password")
        

In [85]:
info = Info()
info.add("Jane")

What is passworddeadbeef
Added user Jane


#### 责任链模式

* 把一个事件/对象/函数，交给一个统一的分发器。谁能处理谁处理

In [96]:
class Event:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return self.name

    
class Widget:
    def __init__(self, parent=None):
        self.parent = parent
    
    def handle(self, event):
        handler = "handle_{}".format(event)
        
        if hasattr(self, handler):
            method = getattr(self, handler)
            return method(event)
        elif self.parent:
            return self.parent.handle(event)
        elif hasattr(self, "handle_default"):
            self.handle_default(event)

In [113]:
class MsgBox(Widget):
    def handle_send_msg(self, event):
        print("call handler", event.name)

class Window(Widget):
    def handle_open(self, event):
        print("call handler", event.name)


In [116]:
msg_box = MsgBox(Window())
event = Event("open")
msg_box.handle(event)

('call handler', 'open')


##### Command Pattern

In [170]:
from abc import abstractmethod
import os
verbose = True


class UndableCommand:

    """Undo more like a common interface
    Better implemented by decorator, rather then inheritance  ??
    """
    
    @abstractmethod
    def execute():
        pass

    @abstractmethod
    def undo():
        pass


class RenameFile(UndableCommand):

    def __init__(self, path_src, path_dest):
        self.src, self.dest = path_src, path_dest

    def execute(self):
        if verbose:
            print("[renaming {} to {}]".format(self.src, self.dest))
        os.rename(self.src, self.dest)

    def undo(self):
        if verbose:
            print("[renaming {} back to {}]".format(self.dest, self.src))

        os.rename(self.dest, self.src)


def delete_file(path):
    if verbose:
        print("[deleting file {}]".format(path))
    os.remove(path)


class CreateFile(UndableCommand):

    def __init__(self, path, txt):
        self.path, self.txt = path, txt

    def execute(self):
        if verbose:
            print("[creating file {}]".format(self.path))
        with open(self.path, "w") as f:
            f.write(self.txt)

    def undo(self):
        delete_file(self.path)


In [171]:
commands = [CreateFile("test", "hello world"),
            CreateFile("test2", "hello world"),
            RenameFile("test", "test3")]

In [172]:
[x.execute() for x in commands]
print("reverse commands sequence ......")
commands.reverse()
[x.undo() for x in commands]

[creating file test]
[creating file test2]
[renaming test to test3]
reverse commands sequence ......
[renaming test3 back to test]
[deleting file test2]
[deleting file test]


[None, None, None]

##### 解释器模式

编写一种DSL(Domain Specific Language), 给专家等不熟悉编程语言的人员一种简单的指令方式

Observer Pattern

一组状态改变时，更新另一组对象

* 一个observer
* 多个对象

逻辑类似MVC，降低发布者，和订阅者之间的耦合度

In [37]:
class Publisher(object):
    def __init__(self):
        self.observers = []
    
    @property
    def data(self):
        return self._data
    
    @data.setter
    def data(self, value):
        self._data = value
    
    
    def add(self, observer):
        if observer not in self.observers:
            self.observers.append(observer)
        else:
            print("Failed to add: {}".format(observer))
            
    def remove(self, observer):
        try:
            self.observers.remove(observer)
        except ValueError:
            print("Failed to remove: {}".format(observer))
        
    def notify(self):
        [o.notify(self) for o in self.observers]
        
class DefaultFormatter(Publisher):
    
    def __init__(self, name):
        super(DefaultFormatter, self).__init__()
        self.name = name
        self._data = 0
    
    def __str__(self):
        return "{}: '{}' has data = {}".format(type(self).__name__, self.name, self._data)

In [38]:
from abc import abstractmethod
class Observer(object):
    @abstractmethod
    def notify(self, *args):
        pass

In [39]:
class BinaryObserver(Observer):
    def notify(self, publisher):
        print("Binary value:", int(bin(publisher.data)[2:]))
        
class HeximalObserver(Observer):
    def notify(self, publisher):
        print("Heximal value:", hex(publisher.data))

In [40]:
formatter = DefaultFormatter("binary")
formatter.add(BinaryObserver())
formatter.add((HeximalObserver()))

In [41]:
formatter.data = 100
formatter.notify()

('Binary value:', 1100100)
('Heximal value:', '0x64')


#### 状态模式

-- state machine，关注状态转换

策略模式

根据不同情况选择最合适的算法，对用户不透明。e.g search, BFS, DFS, 用户调用search，program决定用哪种最合适

模板模式